In [1]:
import os
import glob 
import pandas as pd
from tqdm import tqdm
import time

import warnings
warnings.filterwarnings("ignore")

In [3]:
# Allowing the user to select the dataset so we can locate the respective folder:
user_dataset = input('Enter the dataset to review from the Data Auditor: ')
print("The user would like to use the dataset: ", user_dataset)

The user would like to use the dataset:  aum


In [2]:
# Vehicles that we're interested in are being listed here:

# Core Fixed Income	                    12776	P73285
# Core Plus Fixed Income	            12777	P74285 
# Global Quality Value	                12783	P85285
# Strategic Fixed Income	            12811	P121285
# Strategic Fixed Income Opportunities	12812	P126285
# US Small Cap Core	                    12823	P147285

sheet_names = ['P73285', 'P74285', 'P85285', 'P121285', 'P126285', 'P147285']

In [4]:
# Setting file path. We'll be opening first the Performance folder:
absolute_path = "C:/Users/l.arguello/Downloads/Manulife_DataAuditor/"

file_path = absolute_path+user_dataset

In [5]:
# Using glob to get all the Excel file names in the selected folder, to loop through them:

csv_files = glob.glob(os.path.join(file_path, "*.xlsx")) 
  
file_names = []

# Loop over the list of Excel files: 
for f in tqdm(csv_files, desc="Loading…",ascii=False, ncols=75):

        time.sleep(0.03) 
        # Print the location and filename 
        print('File Name:', f.split("\\")[-1]) 
        # Add each Excel file name to file_names list 
        file_names.append(f.split("\\")[-1])      
 
print("Complete.")

Loading…:   0%|                                     | 0/15 [00:00<?, ?it/s]

File Name: Data_Audit_Report_Albourne_Moatspace_1_2024.xlsx
File Name: Data_Audit_Report_Alpha_Portfolio_Advisors_1_2024.xlsx


Loading…:  47%|█████████████▌               | 7/15 [00:00<00:00, 30.58it/s]

File Name: Data_Audit_Report_Broadridge_1_2024.xlsx
File Name: Data_Audit_Report_Callan_Assoc_1_2024.xlsx
File Name: Data_Audit_Report_Camradata_1_2024.xlsx
File Name: Data_Audit_Report_eVestment_Alliance_1_2024.xlsx
File Name: Data_Audit_Report_Global_Fund_Search_1_2024.xlsx
File Name: Data_Audit_Report_Global_Manager_Research_1_2024.xlsx
File Name: Data_Audit_Report_Investment_Metrics_1_2024.xlsx


Loading…:  73%|████████████████████▌       | 11/15 [00:00<00:00, 29.34it/s]

File Name: Data_Audit_Report_LCG_Assoc_1_2024.xlsx
File Name: Data_Audit_Report_Mercer_1_2024.xlsx
File Name: Data_Audit_Report_Morningstar_1_2024.xlsx
File Name: Data_Audit_Report_Preqin_Hedge_Fund_Analyst_1_2024.xlsx
File Name: Data_Audit_Report_PSN_Informa_1_2024.xlsx


Loading…: 100%|████████████████████████████| 15/15 [00:00<00:00, 29.97it/s]

File Name: Data_Audit_Report_Wilshire_1_2024.xlsx
Complete.


In [6]:
# Loading a sample of the names list obtained from the selected folder:
file_names

['Data_Audit_Report_Albourne_Moatspace_1_2024.xlsx',
 'Data_Audit_Report_Alpha_Portfolio_Advisors_1_2024.xlsx',
 'Data_Audit_Report_Broadridge_1_2024.xlsx',
 'Data_Audit_Report_Callan_Assoc_1_2024.xlsx',
 'Data_Audit_Report_Camradata_1_2024.xlsx',
 'Data_Audit_Report_eVestment_Alliance_1_2024.xlsx',
 'Data_Audit_Report_Global_Fund_Search_1_2024.xlsx',
 'Data_Audit_Report_Global_Manager_Research_1_2024.xlsx',
 'Data_Audit_Report_Investment_Metrics_1_2024.xlsx',
 'Data_Audit_Report_LCG_Assoc_1_2024.xlsx',
 'Data_Audit_Report_Mercer_1_2024.xlsx',
 'Data_Audit_Report_Morningstar_1_2024.xlsx',
 'Data_Audit_Report_Preqin_Hedge_Fund_Analyst_1_2024.xlsx',
 'Data_Audit_Report_PSN_Informa_1_2024.xlsx',
 'Data_Audit_Report_Wilshire_1_2024.xlsx']

In [43]:
# Defining the Excel file to be openned and the sheet we need from the book:
excel_file_orig = pd.read_excel(file_path+'/'+file_names[3], sheet_name=sheet_names[0])
# Printing a file sample:
excel_file_orig.head(3)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 205,Unnamed: 206,Unnamed: 207,Unnamed: 208,Unnamed: 209,Unnamed: 210,Unnamed: 211,Unnamed: 212,Unnamed: 213,Unnamed: 214
0,NaN,NaN,LEGEND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Green Indicates matching data between APX and the reporting database.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Yellow Indicates nonmatching data between APX and the reporting database.,NaN,"*For a data mismatch, information is formated as 'APX data / reporting database data'.",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Selecting the header names placed in row 7:
excel_file_orig.rename(columns = excel_file_orig.iloc[7], inplace= True)
# Selecting the rows with data and reseting the index:
excel_file = excel_file_orig[7:][1:].set_index(['Date'], drop=True)

In [9]:
# Checking data type of all columns in the file:
excel_file.info()
# Date column does not have the correct type, the others are mixed due to characters being in them such as /

<class 'pandas.core.frame.DataFrame'>
Index: 121 entries, 12/2023 to 12/1993
Columns: 214 entries, Corporate - Assets - Total Table to Total Accounts in Range - > $500 Million
dtypes: object(214)
memory usage: 203.2+ KB


In [10]:
# We need information from 09/2022 onwards, so I'll be turning Date column into correct type and then filter by date:
excel_file.index = pd.to_datetime(excel_file.index)
# Selecting data in the dataframe by the correct date:
excel_file = excel_file[~(excel_file.index < '09/2022')]

In [11]:
# Setting up the correct format for the index/Date column
excel_file.index = excel_file.index.strftime("%m/%Y")

In [12]:
# Printing the dataframe with information since 09/2022:
excel_file

,Corporate - Assets - Total Table,Superannuation - Assets - Total Table,Public Fund (Gov) - Assets - Total Table,Union/Multi-Emp - Assets - Total Table,Found&Endow - Assets - Total Table,Health Care - Assets - Total Table,Insurance - Assets - Total Table,High Net Worth - Assets - Total Table,Wrap Accounts - Assets - Total Table,Sub-Advised - Assets - Total Table,...,Total Accounts in Range - $1 Million - $10 Million,Range - $10 Million - $100 Million,Total Assets in Range - $10 Million - $100 Million,Total Accounts in Range - $10 Million - $100 Million,Range - $100 Million - $500 Million,Total Assets in Range - $100 Million - $500 Million,Total Accounts in Range - $100 Million - $500 Million,Range - > $500 Million,Total Assets in Range - > $500 Million,Total Accounts in Range - > $500 Million
Date,,,,,,,,,,,,,,,,,,,,,
12/2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,$10 Million - $100 Million,NaN,NaN,$100 Million - $500 Million,NaN,NaN,> $500 Million,NaN,NaN
09/2023,<NO APX> / $247.21,NaN,<NO APX> / $620.62,NaN,NaN,NaN,<NO APX> / $126.04,NaN,NaN,"<NO APX> / $9,464.74",...,<NO APX> / 1,$10 Million - $100 Million,<NO APX> / $196.73,<NO APX> / 3,$100 Million - $500 Million,<NO APX> / $588.42,<NO APX> / 3,> $500 Million,"<NO APX> / $9,991.49",<NO APX> / 4
06/2023,<NO APX> / $255.22,NaN,<NO APX> / $642.14,NaN,NaN,NaN,<NO APX> / $130.47,NaN,NaN,"<NO APX> / $9,180.89",...,NaN,$10 Million - $100 Million,NaN,NaN,$100 Million - $500 Million,NaN,NaN,> $500 Million,NaN,NaN
03/2023,<NO APX> / $256.93,NaN,<NO APX> / $648.12,NaN,NaN,NaN,<NO APX> / $122.81,NaN,<NO APX> / $175.78,"<NO APX> / $8,760.62",...,NaN,$10 Million - $100 Million,<NO APX> / $125.00,<NO APX> / 3,$100 Million - $500 Million,<NO APX> / $609.18,<NO APX> / 4,> $500 Million,"<NO APX> / $9,229.79",<NO APX> / 3
12/2022,<NO APX> / $499.38,NaN,<NO APX> / $627.40,NaN,NaN,NaN,<NO APX> / $119.03,NaN,<NO APX> / $140.72,"<NO APX> / $8,264.94",...,NaN,$10 Million - $100 Million,<NO APX> / $83.58,<NO APX> / 3,$100 Million - $500 Million,<NO APX> / $829.78,<NO APX> / 5,> $500 Million,"<NO APX> / $8,737.83",<NO APX> / 3
09/2022,<NO APX> / $0.00,<NO APX> / $0.00,<NO APX> / $648.55,<NO APX> / $0.00,<NO APX> / $0.00,<NO APX> / $0.00,<NO APX> / $117.40,<NO APX> / $0.00,<NO APX> / $62.67,"<NO APX> / $8,147.53",...,<NO APX> / 0,$10 Million - $100 Million,<NO APX> / $110.08,<NO APX> / 4,$100 Million - $500 Million,<NO APX> / $230.77,<NO APX> / 2,> $500 Million,"<NO APX> / $8,682.71",<NO APX> / 3


In [13]:
# Dropping rows and columns in which all the cells contain NaN values:
excel_file = excel_file.dropna(how='all', axis=0).dropna(how='all', axis=1)

In [14]:
# Creating a for loop to assign dummy variables to the Data Gap Auditor report:
for n in range(0, excel_file.shape[1]):

    for i,j in enumerate(excel_file[excel_file.columns[(n)]]):
        
        
        try:
            if float(j) >= 0 or float(j) <= 0:

                excel_file[excel_file.columns[n]][i] = excel_file[excel_file.columns[n]][i].replace(j, '0') # "Complete"
                
        except:
            if "<NO APX> / " in j:
                excel_file[excel_file.columns[n]][i] = excel_file[excel_file.columns[n]][i].replace(j, '1') # "Data not in the Vault"
            elif " / <NO DATA>" in j:
                excel_file[excel_file.columns[n]][i] = excel_file[excel_file.columns[n]][i].replace(j, '2') # "Data not in the database"
            elif " / " in j:
                excel_file[excel_file.columns[n]][i] = excel_file[excel_file.columns[n]][i].replace(j, '3') # "Data not matching"  
            else:
                excel_file[excel_file.columns[n]][i] = excel_file[excel_file.columns[n]][i].replace(j, '')  # If the cell does not contain any of this criteria above, then it's not relevant for our analysis/reviewal


In [15]:
# Let's fill the NaN values for easier further processes:
excel_file.fillna('', inplace=True)

In [16]:
excel_file

,Corporate - Assets - Total Table,Superannuation - Assets - Total Table,Public Fund (Gov) - Assets - Total Table,Union/Multi-Emp - Assets - Total Table,Found&Endow - Assets - Total Table,Health Care - Assets - Total Table,Insurance - Assets - Total Table,High Net Worth - Assets - Total Table,Wrap Accounts - Assets - Total Table,Sub-Advised - Assets - Total Table,...,Total Accounts in Range - $1 Million - $10 Million,Range - $10 Million - $100 Million,Total Assets in Range - $10 Million - $100 Million,Total Accounts in Range - $10 Million - $100 Million,Range - $100 Million - $500 Million,Total Assets in Range - $100 Million - $500 Million,Total Accounts in Range - $100 Million - $500 Million,Range - > $500 Million,Total Assets in Range - > $500 Million,Total Accounts in Range - > $500 Million
Date,,,,,,,,,,,,,,,,,,,,,
12/2023,,,,,,,,,,,...,,,,,,,,,,
09/2023,1,,1,,,,1,,,1,...,1,,1,1,,1,1,,1,1
06/2023,1,,1,,,,1,,,1,...,,,,,,,,,,
03/2023,1,,1,,,,1,,1,1,...,,,1,1,,1,1,,1,1
12/2022,1,,1,,,,1,,1,1,...,,,1,1,,1,1,,1,1
09/2022,1,1,1,1,1,1,1,1,1,1,...,1,,1,1,,1,1,,1,1


In [17]:
# Putting the dummy variables in a single column:
excel_file['Review'] = excel_file[excel_file.columns[0:]].apply(lambda x: ''.join(x.astype(str)), axis=1)
# Load a sample of how it looks like at the moment:
excel_file.head()

,Corporate - Assets - Total Table,Superannuation - Assets - Total Table,Public Fund (Gov) - Assets - Total Table,Union/Multi-Emp - Assets - Total Table,Found&Endow - Assets - Total Table,Health Care - Assets - Total Table,Insurance - Assets - Total Table,High Net Worth - Assets - Total Table,Wrap Accounts - Assets - Total Table,Sub-Advised - Assets - Total Table,...,Range - $10 Million - $100 Million,Total Assets in Range - $10 Million - $100 Million,Total Accounts in Range - $10 Million - $100 Million,Range - $100 Million - $500 Million,Total Assets in Range - $100 Million - $500 Million,Total Accounts in Range - $100 Million - $500 Million,Range - > $500 Million,Total Assets in Range - > $500 Million,Total Accounts in Range - > $500 Million,Review
Date,,,,,,,,,,,,,,,,,,,,,
12/2023,,,,,,,,,,,...,,,,,,,,,,00000
09/2023,1,,1,,,,1,,,1,...,,1,1,,1,1,,1,1,1111001111000000000011101110111101111011110000...
06/2023,1,,1,,,,1,,,1,...,,,,,,,,,,1111001111000000000011101110111131111000000000...
03/2023,1,,1,,,,1,,1,1,...,,1,1,,1,1,,1,1,1111111111111111111111111111111111111111111111...
12/2022,1,,1,,,,1,,1,1,...,,1,1,,1,1,,1,1,1111111111111111111111111111111111111111111111...


In [19]:
excel_file['Review']

Date
12/2023                                                00000
09/2023    1111001111000000000011101110111101111011110000...
06/2023    1111001111000000000011101110111131111000000000...
03/2023    1111111111111111111111111111111111111111111111...
12/2022    1111111111111111111111111111111111111111111111...
09/2022    1111111111111111111111111111111111111111111111...
Name: Review, dtype: object

In [21]:
# Creating a for loop to assign the correct description to each period:
for i, j in enumerate(excel_file['Review']):
        
        if all('0' in k for k in j):
            excel_file['Review'][i] = excel_file['Review'][i].replace(j, 'Complete')

        elif all('1' in k for k in j):
            excel_file['Review'][i] = excel_file['Review'][i].replace(j, 'Data not in the Vault')

        elif all('2' in k for k in j):
            excel_file['Review'][i] = excel_file['Review'][i].replace(j, 'Data not in the database')   

        elif all('3' in k for k in j):
            excel_file['Review'][i] = excel_file['Review'][i].replace(j, 'Data not matching') 

In [22]:
excel_file['Review']

Date
12/2023                                             Complete
09/2023    1111001111000000000011101110111101111011110000...
06/2023    1111001111000000000011101110111131111000000000...
03/2023    1111111111111111111111111111111111111111111111...
12/2022    1111111111111111111111111111111111111111111111...
09/2022    1111111111111111111111111111111111111111111111...
Name: Review, dtype: object

In [23]:
# Now we need to continue to put the other conditions:
for i,j in enumerate(excel_file['Review']):

    if (('1' in j) and ('0' in j)):
        excel_file['Review'][i] = excel_file['Review'][i].replace(j, 'Data not in the Vault')
    
    elif (('2' in j) and ('0' in j)):
        excel_file['Review'][i] = excel_file['Review'][i].replace(j, 'Data not in the database')

    elif (('3' in j) and ('0' in j)):
        excel_file['Review'][i] = excel_file['Review'][i].replace(j, 'Data not matching')

    elif (('3' in j) and ('1' in j)):
        excel_file['Review'][i] = excel_file['Review'][i].replace(j, 'Data not in the Vault and not matching')
    
    elif (('2' in j) and ('1' in j)):
        excel_file['Review'][i] = excel_file['Review'][i].replace(j, 'Data not in the Vault and not in the database')
    
    elif (('3' in j) and ('2' in j) and ('1' in j)):
        excel_file['Review'][i] = excel_file['Review'][i].replace(j, 'Data not in the Vault, not matching and not in the database')
    

In [24]:
excel_file['Review']

Date
12/2023                 Complete
09/2023    Data not in the Vault
06/2023    Data not in the Vault
03/2023    Data not in the Vault
12/2022    Data not in the Vault
09/2022    Data not in the Vault
Name: Review, dtype: object

In [25]:
# Creating a list for each of the periods in the Review column:
periods_0 = []
periods_1 = []
periods_2 = []
periods_3 = []

for i,j in enumerate(zip(excel_file['Review'],excel_file.index)):

    if j[0] == 'Complete':
        periods_0.append(j[1])

    elif j[0] == 'Data not in the Vault':
        periods_1.append(j[1])
    
    elif j[0] == 'Data not in the database':
        periods_2.append(j[1])

    elif j[0] == 'Data not matching':
        periods_3.append(j[1])


#0 "Complete"
#1 "Data not in the Vault"
#2 "Data not in the database"
#3 "Data not matching"          

In [27]:
# Sorting the lists by ascending order:
periods_0.sort()
periods_1.sort()
periods_2.sort()
periods_3.sort()

In [28]:
excel_file

,Corporate - Assets - Total Table,Superannuation - Assets - Total Table,Public Fund (Gov) - Assets - Total Table,Union/Multi-Emp - Assets - Total Table,Found&Endow - Assets - Total Table,Health Care - Assets - Total Table,Insurance - Assets - Total Table,High Net Worth - Assets - Total Table,Wrap Accounts - Assets - Total Table,Sub-Advised - Assets - Total Table,...,Range - $10 Million - $100 Million,Total Assets in Range - $10 Million - $100 Million,Total Accounts in Range - $10 Million - $100 Million,Range - $100 Million - $500 Million,Total Assets in Range - $100 Million - $500 Million,Total Accounts in Range - $100 Million - $500 Million,Range - > $500 Million,Total Assets in Range - > $500 Million,Total Accounts in Range - > $500 Million,Review
Date,,,,,,,,,,,,,,,,,,,,,
12/2023,,,,,,,,,,,...,,,,,,,,,,Complete
09/2023,1,,1,,,,1,,,1,...,,1,1,,1,1,,1,1,Data not in the Vault
06/2023,1,,1,,,,1,,,1,...,,,,,,,,,,Data not in the Vault
03/2023,1,,1,,,,1,,1,1,...,,1,1,,1,1,,1,1,Data not in the Vault
12/2022,1,,1,,,,1,,1,1,...,,1,1,,1,1,,1,1,Data not in the Vault
09/2022,1,1,1,1,1,1,1,1,1,1,...,,1,1,,1,1,,1,1,Data not in the Vault


#### **✨Output DataFrame:**

I'll be now creating a dictionary to compile the information and have the output as a dataframe by gathering the description found for the respective periods so that they can be inspected.

In [29]:
# Loading the first sheet "Table of Contents" to obtain information that can be input into the output dataframe:
excel_file_content = pd.read_excel(file_path+'/'+file_names[3])

In [30]:
# A description list is created to put in the final review without considering empty period lists:
description = []

if periods_1 := periods_1: description.append("✔ Data not in the Vault for the periods: {}".format((periods_1)).replace("'",'').replace('[','').replace(']',''))
if periods_2 := periods_2: description.append("✔ Data not in the database for the periods: {}".format((periods_2)).replace("'",'').replace('[','').replace(']',''))
if periods_3 := periods_3: description.append("✔ Data not matching for the periods: {}".format((periods_3)).replace("'",'').replace('[','').replace(']',''))

In [31]:
# Checking a sample of the final description:
description

['✔ Data not in the Vault for the periods: 03/2023, 06/2023, 09/2022, 09/2023, 12/2022']

In [32]:
# Building the dictionary to then transform it into a dataframe:

dict = {'Database': excel_file_content.iloc[4][1],      # Database name e.g. "Wilshire"
        excel_file_orig.iloc[6][1]: description,                  # Product/vehicle name with description of findings e.g. "Core Fixed Income Composite (P73285)"
        }                                               

In [33]:
# Creating a new dataframe that will sum up the findings in the Data Auditor:
output_df = pd.DataFrame([dict])

In [35]:
# Putting each description in a single line (this may duplicate the database name):
output_df0 = output_df.explode(excel_file_orig.iloc[6][1])

# Setting the column width to the max so the whole line can be read:
pd.set_option('display.max_colwidth', -1)

# Sample of the final review:
output_df0

,Database,Core Fixed Income Composite (P73285)
0,Callan Assoc.,"✔ Data not in the Vault for the periods: 03/2023, 06/2023, 09/2022, 09/2023, 12/2022"
